In [122]:
import akshare as ak
import datetime
import pandas as pd
import datetime
import numpy as np
import math

# 获取A股现有的指数信息

In [1]:
# 获取A股现有的指数信息
# https://akshare.akfamily.xyz/data/index/index.html
index_stock_info_df = ak.index_stock_info()
print(index_stock_info_df)

    index_code display_name publish_date
0       000001         上证指数   1991-07-15
1       000002         A股指数   1992-02-21
2       000003         B股指数   1992-02-21
3       000004         工业指数   1993-05-03
4       000005         商业指数   1993-05-03
..         ...          ...          ...
700     399994   中证信息安全主题指数   2015-03-12
701     399995     中证基建工程指数   2015-03-12
702     399996     中证智能家居指数   2014-09-17
703     399997       中证白酒指数   2015-01-21
704     399998       中证煤炭指数   2015-02-13

[705 rows x 3 columns]


# 筛选医药行业相关指数

In [43]:
# 筛选其中只包含“医”的指数，可认为是医药行业相关的指数
yi = index_stock_info_df['display_name'].str.contains('医')
df_yi = index_stock_info_df[yi]

In [64]:
# 重置index
df_yi.reset_index(inplace=True)
del df_yi['index'] # 删除重置后自动创建的index列

C:\Users\Stainer\.conda\envs\pytorch\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [68]:
df_yi

,index_code,名称,publish_date
0,000037,上证医药,2009-01-09
1,000075,医药等权,2010-08-18
2,000109,380医药,2011-06-13
3,000121,医药主题,2011-10-18
4,000808,医药生物,2012-02-17
5,000814,细分医药,2012-04-11
6,000841,800医药,2012-12-21
7,000857,500医药,2013-11-06
8,000913,300医药,2007-07-02
9,000933,中证医药,2009-07-03


# 获取医药行业指数成立以来的年化收益率

In [156]:
# try except else : https://www.cnblogs.com/python-miao/p/14454936.html
# 通过 “历史行情数据-通用” https://akshare.akfamily.xyz/data/index/index.html
# 接口: index_zh_a_hist

#目标地址: http://quote.eastmoney.com/center/hszs.html

#描述: 东方财富网-中国股票指数-行情数据

#限量: 单次返回具体指数指定 period 从 start_date 到 end_date 的之间的近期数据
def obtain_price(yi_merge_df,latest_date):
    '''publist_date_price是指数发布日指数收盘点数'''
    publish_date_price=[]
    '''latest_price是输入查询日期指数收盘点数'''
    latest_price=[]
    for i in range(len(yi_merge_df)):
        symbol=yi_merge_df['index_code'][i]
        '''发布日指数收盘点数'''
        start_date=yi_merge_df['publish_date'][i]
        end_date = start_date
        try:
            index_df = ak.index_zh_a_hist(symbol=symbol, period="daily", start_date=start_date, end_date=end_date) # 该函数用于获取指数发布日的指数点数
        except Exception as err:
            print(err)
            print(symbol)
            publish_date_price.append(0)
            
            
        else:
            if len(index_df)>0:
                publish_date_price.append(index_df['收盘'][0])
            else:
                publish_date_price.append(0)
        
        '''查询日期指数收盘点数'''
        try:
            today_index_df = ak.index_zh_a_hist(symbol=symbol, period="daily", start_date=latest_date, end_date=latest_date) # 该函数用于获取指数发布日的指数点数
        except Exception as err:
            print(err)
            print(symbol)
            latest_price.append(0)
            continue
            
        else:
            if len(today_index_df)>0:
                latest_price.append(today_index_df['收盘'][0])
            else:
                latest_price.append(0)
    
    # 将获取的指数发布日指数点数赋值给publish_date_price
    yi_merge_df['publish_date_price']=publish_date_price
    # 将查询日期赋值为“today”
    yi_merge_df['today']=latest_date
    # 将获取的查询日期指数点数赋值给latest_price
    yi_merge_df['latest_price']=latest_price
    
    # 计算当前日期和指数发布日的时间间隔；为了方便计算，均使用pd.to_datetime对日期进行转换，然后相减，除以365，让其以年的形式呈现
    interval = ((pd.to_datetime(yi_merge_df['today'])-pd.to_datetime(yi_merge_df['publish_date'])).dt.days)/365
    # 计算累计净值
    av = yi_merge_df['latest_price']/yi_merge_df['publish_date_price']
    # 计算年化对数收益率
    ret = np.log(av)/interval
    
    yi_merge_df['interval']=interval
    yi_merge_df['av']=av
    yi_merge_df['ret']=ret
    # 按照年化收益率排序；先按照小数排序，再以百分数%形式显示。不然先以百分数形式显示，再排序，会导致排序混乱。
    yi_merge_df.sort_values(by='ret',ascending=False,inplace=True)
    # 百分数形式显示
    yi_merge_df['ret']=yi_merge_df['ret'].apply(lambda x:format(x,'.2%'))
    
    return yi_merge_df
    


In [157]:
# 输入查询日期
latest_date='2023/1/20'
# 输入查询行业
df_yi_index = obtain_price(df_yi,latest_date)
df_yi_index

('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
000808
'NoneType' object is not subscriptable
399978
'NoneType' object is not subscriptable
399978


C:\Users\Stainer\AppData\Local\Temp\ipykernel_13304\639607314.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yi_merge_df['publish_date_price']=publish_date_price
C:\Users\Stainer\AppData\Local\Temp\ipykernel_13304\639607314.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yi_merge_df['today']=latest_date
C:\Users\Stainer\AppData\Local\Temp\ipykernel_13304\639607314.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,index_code,名称,publish_date,publish_date_price,today,latest_price,interval,av,ret
24,399989,中证医疗指数,2014-10-31,0.00,2023/1/20,11509.61,8.227397,inf,inf%
4,000808,医药生物,2012-02-17,0.00,2023/1/20,11511.84,10.931507,inf,inf%
7,000857,500医药,2013-11-06,0.00,2023/1/20,13185.29,9.210959,inf,inf%
0,000037,上证医药,2009-01-09,1822.58,2023/1/20,7881.14,14.038356,4.324167,10.43%
14,399394,国证医药,2012-10-29,4673.65,2023/1/20,12068.62,10.232877,2.582269,9.27%
15,399441,国证生物医药指数,2015-01-20,1883.44,2023/1/20,3835.02,8.005479,2.036178,8.88%
5,000814,细分医药,2012-04-11,4188.81,2023/1/20,10885.29,10.783562,2.598659,8.86%
17,399647,深医药50,2012-08-06,5053.16,2023/1/20,12372.56,10.463014,2.448480,8.56%
13,399386,1000医药,2011-12-02,3318.91,2023/1/20,8101.38,11.142466,2.440976,8.01%
6,000841,800医药,2012-12-21,4815.27,2023/1/20,10593.12,10.087671,2.199902,7.82%


# 沪深300

In [169]:
# 2005年4月8日正式发布
# 和上证医药对比
start='20050408'
latest='20230120'
hs300_start = ak.index_zh_a_hist(symbol=399300, period="daily", start_date=start, end_date=start)
hs300_latest =ak.index_zh_a_hist(symbol=399300, period="daily", start_date=latest, end_date=latest)

In [170]:
hs300_t =((pd.to_datetime(hs300_latest['日期'])-pd.to_datetime(hs300_start['日期'])).dt.days)/365
hs300_av=hs300_latest['收盘']/hs300_start['收盘']
hs300_ret = (np.log(hs300_av))/hs300_t
hs300_ret

0    0.080194
dtype: float64

In [171]:
# 和上证医药对比
start='20090109'
latest='20230120'
hs300_start = ak.index_zh_a_hist(symbol=399300, period="daily", start_date=start, end_date=start)
hs300_latest =ak.index_zh_a_hist(symbol=399300, period="daily", start_date=latest, end_date=latest)
hs300_t =((pd.to_datetime(hs300_latest['日期'])-pd.to_datetime(hs300_start['日期'])).dt.days)/365
hs300_av=hs300_latest['收盘']/hs300_start['收盘']
hs300_ret = (np.log(hs300_av))/hs300_t
hs300_ret

0    0.055506
dtype: float64

# 以下代码不再需要

In [127]:
# 以下代码不再需要
# 将获取的指数发布日指数点数赋值给publish_date_price
yi_merge_df['publish_date_price']=publish_date_price
# 获取当前日期
#today=datetime.date.today()
# 将当前日期赋值给column today
#yi_merge_df['today']=today
# 计算当前日期和指数发布日的时间间隔；为了方便计算，均使用pd.to_datetime对日期进行转换，然后相减，除以365，让其以年的形式呈现
interval = ((pd.to_datetime(yi_merge_df['today'])-pd.to_datetime(yi_merge_df['publish_date'])).dt.days)/365
av = yi_merge_df['最新价']/yi_merge_df['publish_date_price']
# 计算年化对数收益率
ret = np.log(av)/interval

In [144]:
# 以下代码不再需要
yi_merge_df['interval']=interval
yi_merge_df['av']=av
yi_merge_df['ret']=ret
yi_merge_df.sort_values(by='ret',ascending=False,inplace=True)
yi_merge_df['ret']=yi_merge_df['ret'].apply(lambda x:format(x,'.2%'))
yi_merge_df

,index_code,名称,publish_date,代码,最新价,publish_date_price,today,interval,av,ret
4,000857,500医药,2013-11-06,sh000857,13185.2883,0.00,2023-01-24,9.221918,inf,inf%
0,000037,上证医药,2009-01-09,sh000037,7881.1436,1822.58,2023-01-24,14.049315,4.324169,10.42%
11,399394,国证医药,2012-10-29,sz399394,12068.6240,4673.65,2023-01-24,10.243836,2.582270,9.26%
13,399647,深医药50,2012-08-06,sz399647,12372.5620,5053.16,2023-01-24,10.473973,2.448480,8.55%
10,399386,1000医药,2011-12-02,sz399386,8101.3780,3318.91,2023-01-24,11.153425,2.440976,8.00%
5,000913,300医药,2007-07-02,sh000913,11849.8425,3681.96,2023-01-24,15.575342,3.218352,7.50%
7,000933,中证医药,2009-07-03,sz399933,11250.3480,4074.94,2023-01-24,13.569863,2.760862,7.48%
6,000933,中证医药,2009-07-03,sh000933,11250.3476,4074.94,2023-01-24,13.569863,2.760862,7.48%
2,000109,380医药,2011-06-13,sh000109,12830.0926,5624.52,2023-01-24,11.624658,2.281100,7.09%
3,000121,医药主题,2011-10-18,sh000121,11013.4641,5232.45,2023-01-24,11.276712,2.104839,6.60%
